In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import numpy as np
from sklearn.decomposition import PCA

import types
import copy

Using TensorFlow backend.


In [6]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [7]:
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)
# pd.set_option('display.width', 2000)
# pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.max_colwidth', -1)

In [4]:
db_stacknet_pca = DFDB('../trial/stacknet_pca.pkl', auto_commit=False)
df_trial_stacknet_pca = db_stacknet_pca.select()
df_trial_stacknet_pca['kfold'] = df_trial_stacknet_pca['param'].apply(lambda x: x['kfold'])

In [8]:
df_trial_stacknet_pca.loc[[469]][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','kfold']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,kfold
469,2019-05-27 10:24:12.606868,tune 280,47,1.730152,0.000015,1.778465,0.000249,0.048313,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}"


In [2]:
submission_list = [
    '../submission/submission_mystacknet_562.csv',
    '../submission/submission_mystacknet_841.csv',
    '../submission/submission_mystacknet_pca_469.csv',
                  ]


In [3]:
df_sub0 = pd.read_csv(submission_list[0])

In [4]:
df_sub0.head()

,time_to_failure,seg_id
0,3.451488,seg_00030f
1,5.683240,seg_0012b5
2,5.502084,seg_00184e
3,9.441776,seg_003339
4,7.711720,seg_0042cc


In [5]:
df_sub1 = pd.read_csv(submission_list[1])

In [6]:
df_sub1.head()

,time_to_failure,seg_id
0,3.436247,seg_00030f
1,5.816768,seg_0012b5
2,5.649432,seg_00184e
3,9.384599,seg_003339
4,7.506914,seg_0042cc


In [7]:
df_sub2 = pd.read_csv(submission_list[2])

In [8]:
df_sub2.head()

,time_to_failure,seg_id
0,3.415621,seg_00030f
1,5.510218,seg_0012b5
2,5.450124,seg_00184e
3,9.528377,seg_003339
4,7.456676,seg_0042cc


In [10]:
df_sub = pd.merge(df_sub0, df_sub1, on='seg_id')
df_sub = pd.merge(df_sub, df_sub2, on='seg_id')

In [11]:
df_sub.head()

,time_to_failure_x,seg_id,time_to_failure_y,time_to_failure
0,3.451488,seg_00030f,3.436247,3.415621
1,5.683240,seg_0012b5,5.816768,5.510218
2,5.502084,seg_00184e,5.649432,5.450124
3,9.441776,seg_003339,9.384599,9.528377
4,7.711720,seg_0042cc,7.506914,7.456676


In [14]:
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_sub.drop(columns=['seg_id']).values, axis=1)
df_submit['seg_id'] = df_sub['seg_id']
df_submit.to_csv('../submission/submission_merge.csv', index=False)